# import dependencies

In [247]:
import numpy as np
import pandas as pd
import datetime

In [248]:
from adbrew_core.settings import db
from adbrew_core.db import accessor
from adbrew_core.utils import conversion_rates
from adbrew_core.rule_engine import rule_engine_variables
from adbrew_core.db import bid_management_task

In [177]:
accounts = {}
unique_adgs = []

idx = 0
total_accounts = 300

for account in db.account_details.find({"active":True, "fetch_data":True}):
    record = db.report_records.aggregate([
        {
            "$match": {
                "account_id": str(account["_id"]),
                "dt": {
                    "$gte": datetime.datetime.now() - datetime.timedelta(days=381),
                    "$lte": datetime.datetime.now()
                },
                "record_type":"keywords",
                "segment":None,
                "campaign_type":"sp"
            }
        },
        {
            "$group": {
                "_id": {
                    "accountId":"$account_id",
                    "adGroupId":"$adGroupId",
                    "campaignId":"$campaignId",
                    "date":"$dt"
                }
            }
        }
    ], allowDiskUse=True)
    record = pd.DataFrame(record)
    if record.empty:
        continue
    record["accountId"] = record.apply(lambda x:x["_id"]["accountId"], axis=1)
    record["campaignId"] = record.apply(lambda x:x["_id"]["campaignId"], axis=1)
    record["adGroupId"] = record.apply(lambda x:x["_id"]["adGroupId"], axis=1)
    record["date"] = record.apply(lambda x:x["_id"]["date"], axis=1)
    record.drop(columns=["_id"], inplace=True)
    if record.empty:
        continue
    unique_adgs += [record]
    print(idx, end='\r', flush=True)
    idx+=1
    if idx==total_accounts:
        break
    
unique_adgs = pd.concat(unique_adgs)


In [180]:
df = pd.DataFrame(columns = ["accoundID","no_of_days"])

In [203]:
#unique_adgs["accountId"].unique()

In [182]:
unique_adgs.to_csv("all_account_date.csv")

In [195]:
df  =  pd.read_csv("all_account_date.csv")

In [196]:
df.drop(columns =  "Unnamed: 0",axis = 1,inplace = True)

In [197]:
df

,accountId,campaignId,adGroupId,date
0,5ffdb895b08512986422b08c,63953084720638,205031194777134,2022-07-02
1,5ffdb895b08512986422b08c,53447402597745,83907262803345,2022-10-04
2,5ffdb895b08512986422b08c,244602722806295,232064376697851,2022-03-04
3,5ffdb895b08512986422b08c,15252806899425,149688287548263,2022-11-15
4,5ffdb895b08512986422b08c,116390676982095,151018566086404,2022-07-10
...,...,...,...,...
1168818,63cfc3cf726c7a0b80fdce39,75984766596650,124955822344976,2022-12-22
1168819,63cfc3cf726c7a0b80fdce39,75984766596650,124955822344976,2023-01-09
1168820,63cfc3cf726c7a0b80fdce39,75984766596650,124955822344976,2022-12-26
1168821,63cfc3cf726c7a0b80fdce39,75984766596650,124955822344976,2022-12-02


In [201]:
df['accountId'].nunique()

270

In [208]:
df[df['accountId']=="63cfc3cf726c7a0b80fdce39"]['date'].nunique()

51

In [211]:
dic = {"ac_Id":[],"no_of_days":[]}

In [213]:
for  i in df['accountId'].unique():
    a = df[df['accountId']==i]['date'].nunique()
    dic["ac_Id"].append(i)
    dic['no_of_days'].append(a)

In [216]:
df_days = pd.DataFrame(dic)

In [215]:
# accounts having less than 350 days of data

In [220]:
df_days.to_csv("df_days.csv")

In [229]:
df_days[df_days['no_of_days']>330]

,ac_Id,no_of_days
0,5ffdb895b08512986422b08c,365
1,6013fbaaaadf53eb56518f43,365
4,613b51d1565006d4e2b60924,365
5,615451dfe1e966fb58822ed6,365
6,61b3133728133e8d9e12b44a,365
7,61b7ea05aa9248e907a3a4fe,364
9,61b8929c7f1731fdf842d921,365
10,61e83a4740598119fc405f5f,365
12,61f7720e169a5b91c7df71d3,365
13,6215bded0e0c2c6c8cdff5c1,365


In [239]:
db.account_details.count()

<ipython-input-239-321c6c1c5e64>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  db.account_details.count()


720

In [249]:
accounts = {}
unique_adgs = []

idx = 0
total_accounts = 800

for account in db.account_details.find():
    record = db.report_records.aggregate([
        {
            "$match": {
                "account_id": str(account["_id"]),
                "dt": {
                    "$gte": datetime.datetime.now() - datetime.timedelta(days=381),
                    "$lte": datetime.datetime.now()
                },
                "record_type":"keywords",
                "segment":None,
                "campaign_type":"sp"
            }
        },
        {
            "$group": {
                "_id": {
                    "accountId":"$account_id",
                    "adGroupId":"$adGroupId",
                    "campaignId":"$campaignId",
                    "date":"$dt"
                }
            }
        }
    ], allowDiskUse=True)
    record = pd.DataFrame(record)
    if record.empty:
        continue
    record["accountId"] = record.apply(lambda x:x["_id"]["accountId"], axis=1)
    record["campaignId"] = record.apply(lambda x:x["_id"]["campaignId"], axis=1)
    record["adGroupId"] = record.apply(lambda x:x["_id"]["adGroupId"], axis=1)
    record["date"] = record.apply(lambda x:x["_id"]["date"], axis=1)
    record.drop(columns=["_id"], inplace=True)
    if record.empty:
        continue
    unique_adgs += [record]
    print(idx, end='\r', flush=True)
    idx+=1
    if idx==total_accounts:
        break
    
unique_adgs = pd.concat(unique_adgs)


In [250]:
accounts = {}
unique_adgs = []

idx = 0
total_accounts = 800

for account in db.account_details.find():
    record = db.report_records.aggregate([
        {
            "$match": {
                "account_id": str(account["_id"]),
                "dt": {
                    "$gte": datetime.datetime.now() - datetime.timedelta(days=381),
                    "$lte": datetime.datetime.now()
                },
                "record_type":"keywords",
                "segment":None,
                "campaign_type":"sp"
            }
        },
        {
            "$group": {
                "_id": {
                    "accountId":"$account_id",
                    "adGroupId":"$adGroupId",
                    "campaignId":"$campaignId",
                    "date":"$dt"
                }
            }
        }
    ], allowDiskUse=True)
    record = pd.DataFrame(record)
    if record.empty:
        idx+=1
        continue
    record["accountId"] = record.apply(lambda x:x["_id"]["accountId"], axis=1)
    record["campaignId"] = record.apply(lambda x:x["_id"]["campaignId"], axis=1)
    record["adGroupId"] = record.apply(lambda x:x["_id"]["adGroupId"], axis=1)
    record["date"] = record.apply(lambda x:x["_id"]["date"], axis=1)
    record.drop(columns=["_id"], inplace=True)
    if record.empty:
        idx+=1
        continue
    unique_adgs += [record]
    print(idx, end='\r', flush=True)
    idx+=1
    if idx==total_accounts:
        break
    
unique_adgs = pd.concat(unique_adgs)


In [252]:
accounts_active = {}
unique_adgs_active = []

idx = 0
total_accounts = 800

for account in db.account_details.find({"active":True, "fetch_data":True}):
    record = db.report_records.aggregate([
        {
            "$match": {
                "account_id": str(account["_id"]),
                "dt": {
                    "$gte": datetime.datetime.now() - datetime.timedelta(days=381),
                    "$lte": datetime.datetime.now()
                },
                "record_type":"keywords",
                "segment":None,
                "campaign_type":"sp"
            }
        },
        {
            "$group": {
                "_id": {
                    "accountId":"$account_id",
                    "adGroupId":"$adGroupId",
                    "campaignId":"$campaignId",
                    "date":"$dt"
                }
            }
        }
    ], allowDiskUse=True)
    record = pd.DataFrame(record)
    if record.empty:
        idx+=1
        continue
    record["accountId"] = record.apply(lambda x:x["_id"]["accountId"], axis=1)
    record["campaignId"] = record.apply(lambda x:x["_id"]["campaignId"], axis=1)
    record["adGroupId"] = record.apply(lambda x:x["_id"]["adGroupId"], axis=1)
    record["date"] = record.apply(lambda x:x["_id"]["date"], axis=1)
    record.drop(columns=["_id"], inplace=True)
    if record.empty:
        idx+=1
        continue
    unique_adgs_active += [record]
    print(idx, end='\r', flush=True)
    idx+=1
    if idx==total_accounts:
        break
    
unique_adgs_active = pd.concat(unique_adgs_active)

In [262]:
df = pd.read_csv("all_account_with_date_active.csv")

In [264]:
df.drop(columns = "Unnamed: 0",inplace = True)

In [265]:
df.head()

,accountId,campaignId,adGroupId,date
0,5ffdb895b08512986422b08c,30363257828591,124538790368366,2022-04-18
1,5ffdb895b08512986422b08c,39985714646333,86657809587279,2022-09-16
2,5ffdb895b08512986422b08c,143147470302164,115183069812422,2022-09-10
3,5ffdb895b08512986422b08c,228670193375695,39077164361910,2022-06-12
4,5ffdb895b08512986422b08c,132363851972316,116909827642428,2022-08-22


In [267]:
dic_for_active = {"Id":[],'no_of_days':[]}

In [268]:
for  i in df['accountId'].unique():
    a = df[df['accountId']==i]['date'].nunique()
    dic_for_active["Id"].append(i)
    dic_for_active['no_of_days'].append(a)

In [272]:
df_days_active = pd.DataFrame(dic_for_active)

In [273]:
df_days_active.to_csv("df_days_active.csv")

In [274]:
pd.read_csv("df_days_active.csv")

,Unnamed: 0,Id,no_of_days
0,0,5ffdb895b08512986422b08c,365
1,1,6013fbaaaadf53eb56518f43,365
2,2,612e3e3936240d16135f1944,252
3,3,613b4e3416cb8c3ddcb60925,52
4,4,613b51d1565006d4e2b60924,365
...,...,...,...
265,265,63ca9d9e543578e72cbaab90,57
266,266,63cb891adab9c845ba47c740,65
267,267,63cb9627dab9c845ba47c741,65
268,268,63ce34e69b66eadd5cfdce2e,63


In [276]:
df_days_active[df_days_active['no_of_days'] == 365]

,Id,no_of_days
0,5ffdb895b08512986422b08c,365
1,6013fbaaaadf53eb56518f43,365
4,613b51d1565006d4e2b60924,365
5,615451dfe1e966fb58822ed6,365
6,61b3133728133e8d9e12b44a,365
9,61b8929c7f1731fdf842d921,365
10,61e83a4740598119fc405f5f,365
12,61f7720e169a5b91c7df71d3,365
13,6215bded0e0c2c6c8cdff5c1,365
14,621e270ca49d1163c27d1f49,365


In [280]:
df_days_active[df_days_active['no_of_days'] >329]['Id'].nunique()

21